<a href="https://colab.research.google.com/github/jsn-li/evadb/blob/main/Home_Price_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Home Price Forecasting with EvaDB

## Setup
We must install Postgres and EvaDB and integrate them. Then, we load our database with our home sale data.

### Install and Initialize Postgres

In [ ]:
!apt install postgresql
!service postgresql start
!sudo -u postgres psql -c "CREATE USER admin WITH SUPERUSER PASSWORD 'password'"
!sudo -u postgres psql -c "CREATE DATABASE evadb"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl logrotate netbase
  postgresql-14 postgresql-client-14 postgresql-client-common postgresql-common ssl-cert sysstat
Suggested packages:
  bsd-mailx | mailx postgresql-doc postgresql-doc-14 isag
The following NEW packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl logrotate netbase
  postgresql postgresql-14 postgresql-client-14 postgresql-client-common postgresql-common ssl-cert
  sysstat
0 upgraded, 13 newly installed, 0 to remove and 18 not upgraded.
Need to get 18.3 MB of archives.
After this operation, 51.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 logrotate amd64 3.19.0-1ubuntu1.1 [54.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd6

### Install and Configure EvaDB

In [ ]:
!pip install "evadb[postgres,forecasting] @ git+https://github.com/georgia-tech-db/evadb.git@68265d3b138babfe4a20091bc5fa7a67b56072f5"

In [ ]:
import evadb
cursor = evadb.connect().cursor()

params = {
    "user": "admin",
    "password": "password",
    "host": "localhost",
    "port": "5432",
    "database": "evadb",
}
query = f"CREATE DATABASE postgres_data WITH ENGINE = 'postgres', PARAMETERS = {params};"
cursor.query(query).df()

Downloading: "http://ml.cs.tsinghua.edu.cn/~chenxi/pytorch-models/mnist-b07bb66b.pth" to /root/.cache/torch/hub/checkpoints/mnist-b07bb66b.pth
100%|██████████| 1.03M/1.03M [00:01<00:00, 868kB/s]
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


,0
0,The database postgres_data has been successful...


### Load the Home Sales Dataset


In [ ]:
## drop table if exists
cursor.query("""
  USE postgres_data {
    DROP TABLE IF EXISTS home_sales
  }
""").df()

## create home sale data table
cursor.query("""
  USE postgres_data {
    CREATE TABLE home_sales(date_recorded VARCHAR(64), town VARCHAR(64), sale_amount DECIMAL, property_type VARCHAR(64), residential_type VARCHAR(64))
  }
""").df()

## populate home sale table with dataset
!wget -O /connecticut.csv https://www.dropbox.com/scl/fi/jqkg6xw95k3vxde9mol37/connecticut.csv?rlkey=veomjh3p04oulcpmdkhg4kyta&dl=1
cursor.query("""
  USE postgres_data {
    COPY home_sales(date_recorded, town, sale_amount, property_type, residential_type)
    FROM '/connecticut.csv'
    DELIMITER ',' CSV HEADER
  }
""").df()

--2023-10-18 02:05:20--  https://www.dropbox.com/scl/fi/jqkg6xw95k3vxde9mol37/connecticut.csv?rlkey=veomjh3p04oulcpmdkhg4kyta
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc16f4425d54cf8dd964423553d2.dl.dropboxusercontent.com/cd/0/inline/CFxLpcObIuBeLpsRkxbwg7u5zHY-KMm2AkOHKVhYaiqc4hXljNMy-bwWnznkt6BHRfc5CouhujixD8VNe91jgaKbGJnQztA4uRUEJrfbywMhZzahBTnhYJ0CwCkDGqPeSPfJRKKJ_JmqgFF1QV2dF0xZ/file# [following]
--2023-10-18 02:05:21--  https://uc16f4425d54cf8dd964423553d2.dl.dropboxusercontent.com/cd/0/inline/CFxLpcObIuBeLpsRkxbwg7u5zHY-KMm2AkOHKVhYaiqc4hXljNMy-bwWnznkt6BHRfc5CouhujixD8VNe91jgaKbGJnQztA4uRUEJrfbywMhZzahBTnhYJ0CwCkDGqPeSPfJRKKJ_JmqgFF1QV2dF0xZ/file
Resolving uc16f4425d54cf8dd964423553d2.dl.dropboxusercontent.com (uc16f4425d54cf8dd964423553d2.dl.dropboxusercontent.com)... 162.125.64.15, 26

,status
0,success


## Train the Model using EvaDB


In [ ]:
## Set parameters!
TOWN="Ansonia"  # town to forecast pricing for. valid values: any town in Connecticut
PROPERTY_TYPE="Single Family"  # what type of property to forecast prices for. valid values: "Condo", "Single Family", "Two Family", "Three Family", "Vacant Land", "Commerical", "Apartments"
FREQUENCY="M"  # prediction granularity. valid values: D, W, M, Y (day, week, month, year)
HORIZON=4  # how many predictions to make (e.g. 3 for 3 months (when prediction frequency is set to M))

## Train model
cursor.query(f"""
  CREATE OR REPLACE FUNCTION Forecast FROM
    (
      SELECT town, property_type, residential_type, date_recorded, sale_amount
      FROM postgres_data.home_sales
      WHERE town = '{TOWN}' AND (
        (property_type = '{PROPERTY_TYPE}')
        OR
        (property_type = 'Residential' AND residential_type = '{PROPERTY_TYPE}')
      )
    )
  TYPE Forecasting
  PREDICT 'sale_amount'
  TIME 'date_recorded'
  ID 'town'
  FREQUENCY '{FREQUENCY}'
  HORIZON {HORIZON}
""").df()

Training, please wait...


,0
0,Function Forecast overwritten.


## Retrieve Forecast Result

In [ ]:
cursor.query("SELECT Forecast();").df()

,forecast.town,forecast.date_recorded,forecast.sale_amount
0,Ansonia,2021-10-31,240983.718750
1,Ansonia,2021-11-30,243213.046875
2,Ansonia,2021-12-31,240687.437500
3,Ansonia,2022-01-31,240857.796875
